## <font color="blue">Imports</font>

In [1]:
from google.colab import drive, files # specific to Google Colab
import pandas as pd
import requests
import json
import time

# show all columns
pd.set_option('display.max_columns', None)

## <font color="blue">Local & Constants</font>

In [2]:
############
# OPTIONAL #
############

# mount drive
drive.mount('/content/drive', force_remount=True)

# data location
file_dir = '/content/drive/My Drive/Colab Data/input/' # optional

Mounted at /content/drive


## <font color="green">Zillow</font>

### <font color="purple">Endpoint: Property</font>

In [3]:
search_str = '77008'
# get data
url = "https://zillow-com1.p.rapidapi.com/propertyExtendedSearch"

querystring = {"location":search_str,
               "home_type":"Houses",
               "minPrice": "500000",
               "maxPrice": "2000000",
               "sqftMin": "2500",
               "bedsMin" : "3", 
               }

headers = {
	"X-RapidAPI-Key": "a888d97dddmsh13eee6be68a96c0p13aa9ejsnf528c36dcdc4",
	"X-RapidAPI-Host": "zillow-com1.p.rapidapi.com"
}
z_for_sale_resp = requests.request("GET", url, headers=headers, params=querystring)

# transform to json
z_for_sale_resp_json = z_for_sale_resp.json()
z_for_sale_resp_json

{'props': [{'address': '912 Yale St APT D, Houston, TX 77008',
   'bathrooms': 4,
   'bedrooms': 3,
   'contingentListingType': None,
   'country': 'USA',
   'currency': 'USD',
   'dateSold': None,
   'daysOnZillow': -1,
   'hasImage': True,
   'imgSrc': 'https://photos.zillowstatic.com/fp/e521d5a66cfb2a3de53aa4f6911c26c6-p_e.jpg',
   'latitude': 29.78784,
   'listingStatus': 'FOR_SALE',
   'listingSubType': {'is_FSBA': True},
   'livingArea': 2947,
   'longitude': -95.39863,
   'lotAreaUnit': 'sqft',
   'lotAreaValue': 1611.72,
   'price': 595000,
   'propertyType': 'SINGLE_FAMILY',
   'unit': 'Apt D',
   'zpid': '2100926277'},
  {'address': '1102 Pearlwood Dr, Houston, TX 77008',
   'bathrooms': 3,
   'bedrooms': 3,
   'contingentListingType': None,
   'country': 'USA',
   'currency': 'USD',
   'dateSold': None,
   'daysOnZillow': -1,
   'hasImage': True,
   'imgSrc': 'https://photos.zillowstatic.com/fp/47ecf1ffb1c75a06a8167fa57eb782a4-p_e.jpg',
   'latitude': 29.78949,
   'listingSt

In [4]:
# view data
df_z_for_sale = pd.json_normalize(data=z_for_sale_resp_json['props'])
print('Num of rows:', len(df_z_for_sale))
print('Num of cols:', len(df_z_for_sale.columns))
df_z_for_sale.head()

Num of rows: 40
Num of cols: 24


,dateSold,propertyType,lotAreaValue,address,imgSrc,price,bedrooms,unit,longitude,latitude,listingStatus,zpid,contingentListingType,daysOnZillow,bathrooms,livingArea,country,currency,lotAreaUnit,hasImage,listingSubType.is_FSBA,listingSubType.is_openHouse,newConstructionType,listingSubType.is_newHome
0,None,SINGLE_FAMILY,1611.720,"912 Yale St APT D, Houston, TX 77008",https://photos.zillowstatic.com/fp/e521d5a66cf...,595000,3,Apt D,-95.398630,29.787840,FOR_SALE,2100926277,None,-1,4,2947,USA,USD,sqft,True,True,NaN,NaN,NaN
1,None,SINGLE_FAMILY,2400.156,"1102 Pearlwood Dr, Houston, TX 77008",https://photos.zillowstatic.com/fp/47ecf1ffb1c...,585000,3,NaN,-95.438410,29.789490,FOR_SALE,305346533,None,-1,3,2580,USA,USD,sqft,True,True,True,NaN,NaN
2,None,SINGLE_FAMILY,3001.284,"626 E 26th St, Houston, TX 77008",https://photos.zillowstatic.com/fp/d97b01423a9...,790000,4,NaN,-95.391365,29.810398,FOR_SALE,27784330,None,-1,4,2808,USA,USD,sqft,True,True,NaN,NaN,NaN
3,None,SINGLE_FAMILY,4399.560,"1823 W 14th 1/2 St, Houston, TX 77008",https://photos.zillowstatic.com/fp/0b7311f0c19...,799000,3,NaN,-95.415470,29.796757,FOR_SALE,89556662,None,-1,3,3441,USA,USD,sqft,True,True,NaN,NaN,NaN
4,None,SINGLE_FAMILY,NaN,"627 W 25th St, Houston, TX 77008",https://photos.zillowstatic.com/fp/0e126fa6bb3...,669900,3,NaN,-95.408710,29.808900,FOR_SALE,2125126583,None,-1,3,2576,USA,USD,None,True,NaN,NaN,BUILDER_SPEC,True


In [5]:
# get zpids to a list
zpid_list = df_z_for_sale['zpid'].tolist()
zpid_list

# filter to check 1st 2 zpid
zpid_list = zpid_list[0:2]

In [6]:
# get property detail

# create empty list
prop_detail_list = []

# iterate through list of properties
for zpid in zpid_list:

  # end point
  url = "https://zillow-com1.p.rapidapi.com/property"

  querystring = {"zpid":zpid}

  # header
  headers = {
      'x-rapidapi-host': "zillow-com1.p.rapidapi.com",
      'x-rapidapi-key': "a888d97dddmsh13eee6be68a96c0p13aa9ejsnf528c36dcdc4"
      }

  # get property detail
  z_prop_detail_resp = requests.request("GET", url, headers=headers, params=querystring)
  z_prop_detail_resp_json = z_prop_detail_resp.json()

  # wait 1 sec based on limit
  time.sleep(1.5)

  prop_detail_list.append(z_prop_detail_resp_json)

In [7]:
# convert to dataframe
df_z_prop_detail = pd.json_normalize(prop_detail_list)
print('Num of rows:', len(df_z_prop_detail))
print('Num of cols:', len(df_z_prop_detail.columns))
df_z_prop_detail.head(2)

Num of rows: 2
Num of cols: 255


,listingProvider,buildingPermits,propertyTaxRate,contact_recipients,solarPotential,longitude,countyFIPS,cityId,timeOnZillow,url,zestimate,imgSrc,zpid,zipcode,livingAreaValue,zestimateLowPercent,isListedByOwner,propertyTypeDimension,streetAddress,county,stateId,countyId,timeZone,homeType,livingAreaUnits,comingSoonOnMarketDate,livingArea,bathrooms,annualHomeownersInsurance,state,rentZestimate,building,brokerId,yearBuilt,brokerageName,dateSold,price,pageViewCount,description,homeStatus,homeFacts,latitude,datePosted,bedrooms,nearbyHomes,monthlyHoaFee,priceHistory,favoriteCount,schools,zestimateHighPercent,mlsid,city,providerListingID,country,currency,contingentListingType,resoFacts.hasAttachedProperty,resoFacts.frontageType,resoFacts.poolFeatures,resoFacts.flooring,resoFacts.foundationDetails,resoFacts.accessibilityFeatures,resoFacts.hasGarage,resoFacts.hasPetsAllowed,resoFacts.bodyType,resoFacts.topography,resoFacts.landLeaseExpirationDate,resoFacts.hasAdditionalParcels,resoFacts.waterViewYN,resoFacts.atAGlanceFacts,resoFacts.offerReviewDate,resoFacts.aboveGradeFinishedArea,resoFacts.buyerAgencyCompensationType,resoFacts.rooms,resoFacts.carportSpaces,resoFacts.cityRegion,resoFacts.mainLevelBathrooms,resoFacts.hasPrivatePool,resoFacts.associationFeeIncludes,resoFacts.waterSource,resoFacts.coveredSpaces,resoFacts.inclusions,resoFacts.hasFireplace,resoFacts.zoningDescription,resoFacts.garageSpaces,resoFacts.sewer,resoFacts.elevationUnits,resoFacts.exclusions,resoFacts.mainLevelBedrooms,resoFacts.numberOfUnitsVacant,resoFacts.hasWaterfrontView,resoFacts.listAOR,resoFacts.lotSize,resoFacts.entryLevel,resoFacts.irrigationWaterRightsAcres,resoFacts.greenWaterConservation,resoFacts.stories,resoFacts.livingArea,resoFacts.numberOfUnitsInCommunity,resoFacts.listingTerms,resoFacts.otherStructures,resoFacts.associationFee,resoFacts.marketingType,resoFacts.greenIndoorAirQuality,resoFacts.greenSustainability,resoFacts.hasHomeWarranty,resoFacts.associationPhone,resoFacts.greenBuildingVerificationType,resoFacts.elementarySchool,resoFacts.bedrooms,resoFacts.architecturalStyle,resoFacts.listingId,resoFacts.structureType,resoFacts.interiorFeatures,resoFacts.horseAmenities,resoFacts.electric,resoFacts.developmentStatus,resoFacts.lotFeatures,resoFacts.roofType,resoFacts.waterSources,resoFacts.constructionMaterials,resoFacts.fireplaceFeatures,resoFacts.doorFeatures,resoFacts.appliances,resoFacts.daysOnZillow,resoFacts.bathroomsPartial,resoFacts.bathroomsHalf,resoFacts.depositsAndFees,resoFacts.canRaiseHorses,resoFacts.buildingName,resoFacts.hasAssociation,resoFacts.petsMaxWeight,resoFacts.commonWalls,resoFacts.totalActualRent,resoFacts.additionalParcelsDescription,resoFacts.associationName,resoFacts.allowedPets,resoFacts.communityFeatures,resoFacts.tenantPays,resoFacts.middleOrJuniorSchool,resoFacts.lotSizeDimensions,resoFacts.roadSurfaceType,resoFacts.hasOpenParking,resoFacts.yearBuilt,resoFacts.livingAreaRangeUnits,resoFacts.irrigationWaterRightsYN,resoFacts.bathroomsFull,resoFacts.horseYN,resoFacts.incomeIncludes,resoFacts.highSchool,resoFacts.leaseTerm,resoFacts.elevation,resoFacts.cropsIncludedYN,resoFacts.attic,resoFacts.subdivisionName,resoFacts.windowFeatures,resoFacts.ownership,resoFacts.woodedArea,resoFacts.middleOrJuniorSchoolDistrict,resoFacts.associationPhone2,resoFacts.buyerAgencyCompensation,resoFacts.heating,resoFacts.frontageLength,resoFacts.virtualTourURLUnbranded,resoFacts.associationAmenities,resoFacts.propertySubType,resoFacts.hasCooling,resoFacts.belowGradeFinishedArea,resoFacts.hasElectricOnProperty,resoFacts.foundationArea,resoFacts.zoning,resoFacts.hoaFee,resoFacts.livingAreaRange,resoFacts.basementYN,resoFacts.availabilityDate,resoFacts.waterBodyName,resoFacts.waterfrontFeatures,resoFacts.specialListingConditions,resoFacts.buildingAreaSource,resoFacts.taxAnnualAmount,resoFacts.parking,resoFacts.hasView,resoFacts.virtualTour,resoFacts.securityFeatures,resoFacts.cooling,resoFacts.entryLocation,resoFacts.levels,resoFacts.ownershi

In [8]:
df_z_prop_detail.to_excel('out.xlsx', index= False)

In [ ]:
df_z_prop_detail["hasImage"] = ""
df_z_prop_detail["listingStatus"] = ""
#df_z_prop_detail["listingSubType"] = ""
df_z_prop_detail["lotAreaUnit"] = ""
df_z_prop_detail["lotAreaValue"] = ""
df_z_prop_detail["propertyType"] = ""
#df_z_prop_detail[""] = ""

In [ ]:
for val, rows in df_z_prop_detail.iterrows():
  temp_df = df_z_for_sale[df_z_for_sale["zpid"]== str(rows['zpid'])]
  df_z_prop_detail["hasImage"][val] = temp_df["hasImage"].iat[0]
  df_z_prop_detail["listingStatus"][val] = temp_df["listingStatus"].iat[0]
  #df_z_prop_detail["listingSubType"][val] = temp_df["listingSubType"].iat[0]
  df_z_prop_detail["lotAreaUnit"][val] = temp_df["lotAreaUnit"].iat[0]
  df_z_prop_detail["lotAreaValue"][val] = temp_df["lotAreaValue"].iat[0]
  df_z_prop_detail["propertyType"][val] = temp_df["propertyType"].iat[0]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-

In [ ]:
df_z_prop_detail

,listingProvider,buildingPermits,propertyTaxRate,contact_recipients,solarPotential,longitude,countyFIPS,cityId,timeOnZillow,url,zestimate,imgSrc,zpid,zipcode,livingAreaValue,zestimateLowPercent,isListedByOwner,propertyTypeDimension,streetAddress,county,stateId,countyId,timeZone,homeType,livingAreaUnits,comingSoonOnMarketDate,livingArea,bathrooms,annualHomeownersInsurance,state,rentZestimate,building,brokerId,yearBuilt,brokerageName,dateSold,price,pageViewCount,description,homeStatus,homeFacts,latitude,datePosted,bedrooms,nearbyHomes,monthlyHoaFee,priceHistory,favoriteCount,schools,zestimateHighPercent,mlsid,city,providerListingID,country,currency,contingentListingType,resoFacts.hasAttachedProperty,resoFacts.frontageType,resoFacts.poolFeatures,resoFacts.flooring,resoFacts.foundationDetails,resoFacts.accessibilityFeatures,resoFacts.hasGarage,resoFacts.hasPetsAllowed,resoFacts.bodyType,resoFacts.topography,resoFacts.landLeaseExpirationDate,resoFacts.hasAdditionalParcels,resoFacts.waterViewYN,resoFacts.atAGlanceFacts,resoFacts.offerReviewDate,resoFacts.aboveGradeFinishedArea,resoFacts.buyerAgencyCompensationType,resoFacts.rooms,resoFacts.carportSpaces,resoFacts.cityRegion,resoFacts.mainLevelBathrooms,resoFacts.hasPrivatePool,resoFacts.associationFeeIncludes,resoFacts.waterSource,resoFacts.coveredSpaces,resoFacts.inclusions,resoFacts.hasFireplace,resoFacts.zoningDescription,resoFacts.garageSpaces,resoFacts.sewer,resoFacts.elevationUnits,resoFacts.exclusions,resoFacts.mainLevelBedrooms,resoFacts.numberOfUnitsVacant,resoFacts.hasWaterfrontView,resoFacts.listAOR,resoFacts.lotSize,resoFacts.entryLevel,resoFacts.irrigationWaterRightsAcres,resoFacts.greenWaterConservation,resoFacts.stories,resoFacts.livingArea,resoFacts.numberOfUnitsInCommunity,resoFacts.listingTerms,resoFacts.otherStructures,resoFacts.associationFee,resoFacts.marketingType,resoFacts.greenIndoorAirQuality,resoFacts.greenSustainability,resoFacts.hasHomeWarranty,resoFacts.associationPhone,resoFacts.greenBuildingVerificationType,resoFacts.elementarySchool,resoFacts.bedrooms,resoFacts.architecturalStyle,resoFacts.listingId,resoFacts.structureType,resoFacts.interiorFeatures,resoFacts.horseAmenities,resoFacts.electric,resoFacts.developmentStatus,resoFacts.lotFeatures,resoFacts.roofType,resoFacts.waterSources,resoFacts.constructionMaterials,resoFacts.fireplaceFeatures,resoFacts.doorFeatures,resoFacts.appliances,resoFacts.daysOnZillow,resoFacts.bathroomsPartial,resoFacts.bathroomsHalf,resoFacts.depositsAndFees,resoFacts.canRaiseHorses,resoFacts.buildingName,resoFacts.hasAssociation,resoFacts.petsMaxWeight,resoFacts.commonWalls,resoFacts.totalActualRent,resoFacts.additionalParcelsDescription,resoFacts.associationName,resoFacts.allowedPets,resoFacts.communityFeatures,resoFacts.tenantPays,resoFacts.middleOrJuniorSchool,resoFacts.lotSizeDimensions,resoFacts.roadSurfaceType,resoFacts.hasOpenParking,resoFacts.yearBuilt,resoFacts.livingAreaRangeUnits,resoFacts.irrigationWaterRightsYN,resoFacts.bathroomsFull,resoFacts.horseYN,resoFacts.incomeIncludes,resoFacts.highSchool,resoFacts.leaseTerm,resoFacts.elevation,resoFacts.cropsIncludedYN,resoFacts.attic,resoFacts.subdivisionName,resoFacts.windowFeatures,resoFacts.ownership,resoFacts.woodedArea,resoFacts.middleOrJuniorSchoolDistrict,resoFacts.associationPhone2,resoFacts.buyerAgencyCompensation,resoFacts.heating,resoFacts.frontageLength,resoFacts.virtualTourURLUnbranded,resoFacts.associationAmenities,resoFacts.propertySubType,resoFacts.hasCooling,resoFacts.belowGradeFinishedArea,resoFacts.hasElectricOnProperty,resoFacts.foundationArea,resoFacts.zoning,resoFacts.hoaFee,resoFacts.livingAreaRange,resoFacts.basementYN,resoFacts.availabilityDate,resoFacts.waterBodyName,resoFacts.waterfrontFeatures,resoFacts.specialListingConditions,resoFacts.buildingAreaSource,resoFacts.taxAnnualAmount,resoFacts.parking,resoFacts.hasView,resoFacts.virtualTour,resoFacts.securityFeatures,resoFacts.cooling,resoFacts.entryLocation,resoFacts.levels,resoFacts.ownershi

In [ ]:
for val1, rows1 in df_z_prop_detail.iterrows():
  #print(rows1)
  if rows1["lotAreaUnit"] =="acres":
    df_z_prop_detail["lotAreaUnit"][val1] = int(rows1["lotAreaValue"]) * 43560

In [ ]:
final_df = df_z_prop_detail[(df_z_prop_detail['lotAreaValue'] >= 4500) & (df_z_prop_detail['resoFacts.hoaFee'] == None)]

if len(final_df) <1:
  print("There were no property with the given filters.")

There were no property with the given filters.


In [ ]:
# columns of interest placeholder for now
detail_cols = ['streetAddress', 
 'city',
 'county',
 'zipcode',
 'state',
 'price',
 'homeType',
 'timeOnZillow', 
 'zestimate',
 'rentZestimate',
 'livingArea',
 'bedrooms',
 'bathrooms',
 'yearBuilt',
 'description',
 'priceHistory',
 'taxHistory',
 'zpid'
 ]

# retain limited columns for output
df_z_prop_detail_output = df_z_prop_detail[detail_cols]
df_z_prop_detail_output.head()

,streetAddress,city,county,zipcode,state,price,homeType,timeOnZillow,zestimate,rentZestimate,livingArea,bedrooms,bathrooms,yearBuilt,description,priceHistory,taxHistory,zpid
0,912 Yale St APT D,Houston,None,77008,TX,595000,SINGLE_FAMILY,15 days,588670,3534,2947,3,4,2014,"Gorgeous, well-maintained Houston Heights 4-st...","[{'priceChangeRate': 0, 'date': '2022-06-30', ...",NaN,2100926277
1,1102 Pearlwood Dr,Houston,None,77008,TX,585000,SINGLE_FAMILY,7 days,585001,3694,2580,3,3,2019,Located in the premier gated community of Timb...,"[{'priceChangeRate': 0, 'date': '2022-07-08', ...","[{'time': 1626347792719, 'valueIncreaseRate': ...",305346533


In [ ]:
# download file
df_z_prop_detail_output.to_csv(str(search_str) + '_prop_detail_output.csv')
#files.download('df_z_prop_detail_output.csv')

# End Notebook